In [ ]:
!pip install igraph geopandas shapely keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 11.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create the adjacency matrix based on the network distance

In [ ]:
import pandas as pd
import geopandas as gpd
import igraph as ig
import numpy as np
from shapely.geometry import Point, LineString, MultiLineString
from scipy.sparse import lil_matrix

In [ ]:
# Load the shapefile
shapefile_path = '/content/drive/MyDrive/traffic predict/traffic_flow_data/20211110_glasgow_road_link.shp'

# Load the shapefile and check for multi-part geometries
road = gpd.read_file(shapefile_path)
road = road[['TOID','startNode','endNode','SHAPE_Leng','geometry']]

In [ ]:
sensor_locations = pd.read_csv('/content/drive/MyDrive/traffic predict/traffic_flow_data/locations.csv')

In [ ]:
print(road.loc[794].geometry)
print(road.loc[793].geometry)
len(road)

MULTILINESTRING Z ((251248.17809751772 669639.0000000012 6.399999999994179, 251247.0000000005 669639.0000000012 6.399999999994179, 251245.95858065385 669639.2603548367 6.399999999994179), (251238.56264667233 669639.112529336 6.399999999994179, 251238.00000000026 669639.0000000012 6.399999999994179, 251230.00000000055 669635.3069999991 6.5, 251179.45799999978 669611.9780000006 6.69999999999709, 251174.4590000003 669607.2569999994 6.69999999999709, 251170.01599999957 669602.3509999997 6.69999999999709), (251256.00000000073 669631.9999999998 6.399999999994179, 251254.49207804582 669634.0105626032 6.399999999994179))
LINESTRING Z (266867.00000000035 664490.0000000007 60.10000000000582, 266914.00000000023 664422.9999999993 59.69999999999709, 266922 664409.9999999993 59.80000000000291)


32179

In [ ]:
# Reconstruct the road geodataframe to make sure each row as a single edge of the road network
# split a LineString or MultiLineString into two-point LineStrings
def split_to_edges(geometry):
    """Splits a LineString or MultiLineString geometry into individual two-point LineStrings."""
    edges = []

    if isinstance(geometry, LineString):
        # Split LineString into two-point segments
        points = list(geometry.coords)
        for i in range(len(points) - 1):
            edges.append(LineString([points[i], points[i+1]]))

    elif isinstance(geometry, MultiLineString):
        # First break MultiLineString into individual LineStrings
        for line in geometry.geoms:
            # Then split each LineString into two-point segments
            points = list(line.coords)
            for i in range(len(points) - 1):
                edges.append(LineString([points[i], points[i+1]]))

    return edges

# Create a new list to store the two-point LineStrings
new_geometries = []

# Iterate over each row in the GeoDataFrame and split the geometries
for geom in road.geometry:
    new_geometries.extend(split_to_edges(geom))

# Create a new GeoDataFrame from the split geometries
all_edges = gpd.GeoDataFrame(geometry=new_geometries, crs=road.crs)

# Display the first few rows of the new GeoDataFrame
all_edges.head(10)


,geometry
0,"LINESTRING Z (265498 666414 91.5, 265498 66641..."
1,"LINESTRING Z (265498 666416 91.5, 265491.164 6..."
2,"LINESTRING Z (252430 668308 16.1, 252401 66825..."
3,"LINESTRING Z (252401 668254 15.5, 252398.6 668..."
4,"LINESTRING Z (252398.6 668250 15.4, 252389 668..."
5,"LINESTRING Z (252389 668234 14.5, 252369 66820..."
6,"LINESTRING Z (252369 668208 12.8, 252348 66817..."
7,"LINESTRING Z (252348 668177 10.9, 252306 66812..."
8,"LINESTRING Z (252066.564 660968.5 27.9, 252075..."
9,"LINESTRING Z (252075.98 660958.817 28.1, 25208..."


In [ ]:
len(all_edges)

143751

In [ ]:
# Step 1: Convert sensor locations to GeoDataFrame with Point geometries
sensor_locations['geometry'] = sensor_locations.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
sensor_gdf = gpd.GeoDataFrame(sensor_locations, geometry='geometry', crs='EPSG:4326')  # Assuming WGS84 coordinate system

# Convert road edges and sensor locations to the same projection if necessary (road_edges may use a different CRS)
sensor_gdf = sensor_gdf.to_crs(road.crs)

In [ ]:
# Step 2: Calculate the perpendicular distance from each sensor to each edge in the road network
# We'll loop through each sensor and find the nearest edge by minimizing the perpendicular distance

nearest_edges = []  # This will store the nearest edge for each sensor

# Loop through each sensor and find the nearest edge by perpendicular distance
for sensor in sensor_gdf.geometry:
    distances = all_edges.geometry.apply(lambda edge: sensor.distance(edge))  # Perpendicular distance to each edge
    nearest_edge_idx = np.argmin(distances)  # Index of the nearest edge
    nearest_edges.append(all_edges.iloc[nearest_edge_idx])  # Store the nearest edge

# Create a GeoDataFrame with the nearest edges corresponding to each sensor
sensor_to_edge_mapping = gpd.GeoDataFrame(nearest_edges, geometry='geometry')

# Step 3: Filter the unique edges that have sensors mapped to them
sensor_edges = sensor_to_edge_mapping.drop_duplicates()


In [ ]:
len(sensor_edges)

444

In [ ]:
# Step 4: Prepare for calculating road network distances between the mapped sensor edges
# Create a list of all vertices (start and end points of the edges)
vertices = {}  # To map points (start, end) to unique IDs
vertex_id = 0

edges = []  # To store the edges as (start_vertex, end_vertex, distance)
for idx, row in all_edges.iterrows():
    line = row.geometry
    start_point = Point(line.coords[0])
    end_point = Point(line.coords[-1])

    if start_point not in vertices:
        vertices[start_point] = vertex_id
        vertex_id += 1
    if end_point not in vertices:
        vertices[end_point] = vertex_id
        vertex_id += 1

    # Add the edge (start vertex, end vertex, length of the road segment)
    edge_length = start_point.distance(end_point)  # Assuming Euclidean distance for the edge length
    edges.append((vertices[start_point], vertices[end_point], edge_length))

In [ ]:
# Step 5: Create an igraph Graph object for the full road network
g = ig.Graph()
g.add_vertices(len(vertices))  # Add vertices to the graph (number of unique points in the network)
g.add_edges([(e[0], e[1]) for e in edges])  # Add the edges (start_vertex, end_vertex) to the graph
g.es['weight'] = [e[2] for e in edges]  # Set the edge weights as road segment lengths (distances)

In [ ]:
# Step 6: Function to calculate the shortest path between the start vertices of two sensor edges
def get_dynamic_network_distance(edge1, edge2):
    start_v1 = edge1['geometry'].coords[0]  # Start vertex of edge 1
    start_v2 = edge2['geometry'].coords[0]  # Start vertex of edge 2

    # Retrieve the corresponding vertex IDs from the vertices dictionary
    vertex_id_1 = vertices[Point(start_v1)]
    vertex_id_2 = vertices[Point(start_v2)]

    # Use Dijkstra's algorithm to compute the shortest path between the two vertices dynamically
    distance = g.distances(source=vertex_id_1, target=vertex_id_2, weights='weight')[0][0]

    return distance

In [ ]:
# Step 7: Compute the road network distances between each pair of sensor edges dynamically
num_sensors = len(sensor_edges)
sensor_adjacency_matrix = lil_matrix((num_sensors, num_sensors), dtype=np.float32)

for i in range(num_sensors):
    for j in range(i + 1, num_sensors):  # Only compute for upper triangle to avoid redundant calculations
        print(i,j)
        edge1 = sensor_edges.iloc[i]
        edge2 = sensor_edges.iloc[j]

        # Get the shortest road network distance between the start vertices of the sensor edges
        distance = get_dynamic_network_distance(edge1, edge2)

        # Fill the adjacency matrix symmetrically
        sensor_adjacency_matrix[i, j] = distance
        sensor_adjacency_matrix[j, i] = distance  # Symmetry since the graph is undirected

# Now sensor_adjacency_matrix contains the road network distances between all sensor edges

Streaming output truncated to the last 5000 lines.
343 394
343 395
343 396
343 397
343 398
343 399
343 400
343 401
343 402
343 403
343 404
343 405
343 406
343 407
343 408
343 409
343 410
343 411
343 412
343 413
343 414
343 415
343 416
343 417
343 418
343 419
343 420
343 421
343 422
343 423
343 424
343 425
343 426
343 427
343 428
343 429
343 430
343 431
343 432
343 433
343 434
343 435
343 436
343 437
343 438
343 439
343 440
343 441
343 442
343 443
344 345
344 346
344 347
344 348
344 349
344 350
344 351
344 352
344 353
344 354
344 355
344 356
344 357
344 358
344 359
344 360
344 361
344 362
344 363
344 364
344 365
344 366
344 367
344 368
344 369
344 370
344 371
344 372
344 373
344 374
344 375
344 376
344 377
344 378
344 379
344 380
344 381
344 382
344 383
344 384
344 385
344 386
344 387
344 388
344 389
344 390
344 391
344 392
344 393
344 394
344 395
344 396
344 397
344 398
344 399
344 400
344 401
344 402
344 403
344 404
344 405
344 406
344 407
344 408
344 409
344 410
344 411
344 412
344 4

In [ ]:
dense_matrix = sensor_adjacency_matrix.toarray()

In [ ]:
# Save the dense matrix as a CSV file
np.savetxt('/content/drive/MyDrive/traffic predict/dense_matrix.csv', dense_matrix, delimiter=',')

# **Data pre-proess**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

Sensors:

GD030A_S, GD025A_B, GD0451_S

In [ ]:
traffic_data = pd.read_csv('/content/drive/MyDrive/traffic predict/traffic_flow_data/flows/GD030A_S.csv')

## 1. Recover timestamp

In [ ]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

In [ ]:
# Apply the recover_timestamp function to recover the full time series
traffic_full = recover_timestamp(traffic_data)
traffic_full

,date,time,flow
2019-10-01 00:00:00,2019-10-01,0.0,15.0
2019-10-01 01:00:00,2019-10-01,1.0,9.0
2019-10-01 02:00:00,2019-10-01,2.0,9.0
2019-10-01 03:00:00,2019-10-01,3.0,7.0
2019-10-01 04:00:00,2019-10-01,4.0,9.0
...,...,...,...
2023-09-30 19:00:00,2023-09-30,19.0,129.0
2023-09-30 20:00:00,2023-09-30,20.0,119.0
2023-09-30 21:00:00,2023-09-30,21.0,106.0
2023-09-30 22:00:00,2023-09-30,22.0,88.0


In [ ]:
import plotly.graph_objects as go
# Create interactive plot using Plotly
fig = go.Figure()

# Add observed data to the plot
fig.add_trace(go.Scatter(x=traffic_full['flow'].index, y=traffic_full['flow'], mode='lines', name='Observed Data'))

# Update layout for better visualization
fig.update_layout(
    title='Observed Data ',
    xaxis_title='Date',
    yaxis_title='Traffic Flow',
    legend_title='Legend'
)

# Show the plot
fig.show()

## 2. Train, validate, test data split

In [ ]:
train_set = traffic_full[:'2022-02-28 23:00:00']
valid_set = traffic_full['2022-03-01 00:00:00':'2022-12-31 23:00:00']
test_set = traffic_full['2023-01-01 00:00:00':]
print('Proportion of train_set : {:.4f}'.format(len(train_set)/len(traffic_full)))
print('Proportion of valid_set : {:.4f}'.format(len(valid_set)/len(traffic_full)))
print('Proportion of test_set : {:.4f}'.format(len(test_set)/len(traffic_full)))

Proportion of train_set : 0.6037
Proportion of valid_set : 0.2094
Proportion of test_set : 0.1869


## 3. Split the data into X and y



In [ ]:
# Define the create_multi_step_sequence function
def create_multi_step_sequence(data, last_n_steps, day_lag, week_lag, n_future_steps):
    """
    Create input sequences from data using multiple time windows and multiple future steps as output.

    Parameters:
    - data: The time series data (1D array or list with possible NaN values)
    - last_n_steps: Number of most recent steps to use as part of the input (default 12)
    - day_lag: Time lag for the last day's value (24 hours ago, default 24)
    - week_lag: Time lag for the last week's value (168 hours ago, default 168)
    - n_future_steps: Number of future steps to predict (default 6)

    Returns:
    - X: Input features combining last_n_steps, last day's value, and last week's value
    - y: Output labels (shape: [samples, n_future_steps])
    """
    X, y = [], []

    # Loop over the data to create the input-output pairs
    for i in range(max(last_n_steps, day_lag, week_lag), len(data) - n_future_steps):
        # Input sequence of the last `last_n_steps` observations
        input_seq = data['flow'].values[i - last_n_steps:i]

        # Last day's observation (24 hours ago)
        last_day_value = data['flow'].values[i - day_lag]

        # Last week's observation (168 hours ago)
        last_week_value = data['flow'].values[i - week_lag]

        # Output (next `n_future_steps` observations)
        output_seq = data['flow'].values[i:i + n_future_steps]

        # Check if any NaN values exist in the input or output sequences
        if not np.isnan(input_seq).any() and not np.isnan(last_day_value) and not np.isnan(last_week_value) and not np.isnan(output_seq).any():
            # Combine the features: last_n_steps, last_day_value, last_week_value
            X.append(np.concatenate([input_seq, [last_day_value], [last_week_value]]))
            y.append(output_seq)

    # Convert to numpy arrays and reshape X to match CNN expected input (samples, timesteps, features)
    X = np.array(X).reshape(-1, last_n_steps + 2, 1)  # Add the 2 additional features (last_day_value, last_week_value)
    y = np.array(y).reshape(-1, n_future_steps)  # Multiple output steps

    # Convert the entire input and output into a pandas DataFrame with appropriate column names for multi-step prediction

    # Define column names for the input DataFrame
    input_columns = [f'Step_{i}_back' for i in range(last_n_steps, 0, -1)] + ['Day_1_back', 'Week_1_back']

    # Create a DataFrame for the input (all rows)
    df_X = pd.DataFrame(X.reshape(X.shape[0], 14), columns=input_columns)

    # Create a DataFrame for the output (all rows), where each column represents one of the next 6 steps
    output_columns = [f'Next_Step_{i}' for i in range(0, n_future_steps)]
    df_y = pd.DataFrame(y, columns=output_columns)

    return X, y, df_X, df_y

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step) and 5 steps ahead

In [ ]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=6)

In [ ]:
X_train.shape

(19495, 14, 1)

## 4. Normalise the data after split

Normalise X

In [ ]:
# Separate scalers for inputs and outputs
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape x_train to 2D for scaling
n_samples, n_timesteps, n_features = X_train.shape
x_train_reshaped = X_train.reshape(-1, n_features)  # Shape: (n_samples * n_timesteps, n_features)
# Fit the scaler on the training data
x_scaler.fit(x_train_reshaped)
# Transform the training data
x_train_scaled = x_scaler.transform(x_train_reshaped)
# Reshape back to original shape
x_train_scaled = x_train_scaled.reshape(n_samples, n_timesteps, n_features)

# x_val
n_val_samples = X_valid.shape[0]
x_val_reshaped = X_valid.reshape(-1, n_features)
x_val_scaled = x_scaler.transform(x_val_reshaped)
x_val_scaled = x_val_scaled.reshape(n_val_samples, n_timesteps, n_features)

# x_test
n_test_samples = X_test.shape[0]
x_test_reshaped = X_test.reshape(-1, n_features)
x_test_scaled = x_scaler.transform(x_test_reshaped)
x_test_scaled = x_test_scaled.reshape(n_test_samples, n_timesteps, n_features)

Normalise y

In [ ]:
# Reshape y_train to 2D for scaling
y_train_reshaped = y_train.reshape(-1, 1)  # Shape: (n_samples * n_outputs, 1)
# Fit the scaler on the training data
y_scaler.fit(y_train_reshaped)
# Transform the training data
y_train_scaled = y_scaler.transform(y_train_reshaped)
# Reshape back to original shape
y_train_scaled = y_train_scaled.reshape(n_samples, y_train.shape[1])

# y_val
y_val_reshaped = y_valid.reshape(-1, 1)
y_val_scaled = y_scaler.transform(y_val_reshaped)
y_val_scaled = y_val_scaled.reshape(n_val_samples, y_valid.shape[1])

# y_test
y_test_reshaped = y_test.reshape(-1, 1)
y_test_scaled = y_scaler.transform(y_test_reshaped)
y_test_scaled = y_test_scaled.reshape(n_test_samples, y_test.shape[1])

## 5. Define the CNN model

In [ ]:
def create_cnn_model(input_shape, n_outputs, dropout_rate=0.5, learning_rate=0.001, filters=64, kernel_size=3):
    model = keras.Sequential([
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape),
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(dropout_rate),
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(n_outputs)
    ])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
n_outputs = y_train.shape[1]

## 6. Set Up Hyperparameter Grid

In [ ]:
from itertools import product

# Define hyperparameter options
dropout_rates = [0.3, 0.5, 0.7]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]
filters_list = [32, 64, 128]
kernel_sizes = [2, 3, 5]

# Create a list of all hyperparameter combinations
hyperparameter_combinations = list(product(dropout_rates, learning_rates, batch_sizes, filters_list, kernel_sizes))

print(f"Total combinations: {len(hyperparameter_combinations)}")

Total combinations: 243


## 7. Train the Model with Hyperparameter Tuning

In [ ]:
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (dropout_rate, learning_rate, batch_size, filters, kernel_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}, filters={filters}, kernel_size={kernel_size}")

    model = create_cnn_model(
        input_shape, n_outputs,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate,
        filters=filters,
        kernel_size=kernel_size
    )

    # Initialize EarlyStopping inside the loop
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,  # Set higher epochs due to Early Stopping
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0  # Change to 1 for detailed output
    )

    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'filters': filters,
            'kernel_size': kernel_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")



Combination 1/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=32, kernel_size=2


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 6.
Validation MAE: 0.0586

Combination 2/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=32, kernel_size=3
Restoring model weights from the end of the best epoch: 34.
Validation MAE: 0.0575

Combination 3/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=32, kernel_size=5
Epoch 26: early stopping
Restoring model weights from the end of the best epoch: 16.
Validation MAE: 0.0585

Combination 4/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=64, kernel_size=2
Epoch 15: early stopping
Restoring model weights from the end of the best epoch: 5.
Validation MAE: 0.0584

Combination 5/243
Training with dropout_rate=0.3, learning_rate=0.01, batch_size=32, filters=64, kernel_size=3
Epoch 27: early stopping
Restoring model weights from the end of the best epoch: 17.
Validation MAE: 0.0574

Combination 6/243
Training with dropou

## 8. Make Predictions and Get Evaluation Metrics
I forget the store the results of best_model, but still have the results of best_hyperparameter

So I have to recreate the model using these hyperparameters and retrain it

#### 8.1 Recreate the model

In [ ]:
def create_best_cnn_model(input_shape, n_outputs):
    model = keras.Sequential([
        # First convolutional layer
        layers.Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=input_shape),
        # Second convolutional layer
        layers.Conv1D(filters=128, kernel_size=5, activation='relu'),
        # Pooling layer
        layers.MaxPooling1D(pool_size=2),
        # Regularization
        layers.Dropout(0.3),
        # Flattening the output
        layers.Flatten(),
        # Fully connected layers
        layers.Dense(50, activation='relu'),
        layers.Dropout(0.3),
        # Output layer
        layers.Dense(n_outputs)
    ])
    # Compile the model
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        loss='mse',
        optimizer=optimizer,
        metrics=['mae']
    )
    return model

In [ ]:
# Recreate the model
input_shape = (x_train_scaled.shape[1], x_train_scaled.shape[2])
n_outputs = y_train.shape[1]
best_model = create_best_cnn_model(input_shape, n_outputs)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### 8.2 Retrain the model

In [ ]:
# Initialize EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_mae',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
# Train the model
history = best_model.fit(
    x_train_scaled, y_train_scaled,
    epochs=50,  # You can adjust this as needed
    batch_size=64,
    validation_data=(x_val_scaled, y_val_scaled),
    callbacks=[early_stopping],
    verbose=1  # Set to 1 to see detailed training output
)

Epoch 1/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - loss: 0.0264 - mae: 0.1218 - val_loss: 0.0072 - val_mae: 0.0583
Epoch 2/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0117 - mae: 0.0809 - val_loss: 0.0071 - val_mae: 0.0575
Epoch 3/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0102 - mae: 0.0753 - val_loss: 0.0070 - val_mae: 0.0555
Epoch 4/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0096 - mae: 0.0728 - val_loss: 0.0070 - val_mae: 0.0537
Epoch 5/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0094 - mae: 0.0723 - val_loss: 0.0072 - val_mae: 0.0589
Epoch 6/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0092 - mae: 0.0713 - val_loss: 0.0068 - val_mae: 0.0551
Epoch 7/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0090 - mae: 0.0701 - val_loss: 0.0069 - val_mae: 0.0573
Epoch 8/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0087 - mae: 0.0692 - val_loss: 0.0071 - val_mae: 0.0587
Epoch 9/50
305/305 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - l

#### 8.3 Make Predictions and Get Evaluation Metrics

In [ ]:
# Make predictions
y_pred_scaled = best_model.predict(x_test_scaled)

# Reshape for inverse scaling
y_pred_reshaped = y_pred_scaled.reshape(-1, 1)
y_test_reshaped = y_test_scaled.reshape(-1, 1)

# Inverse transform
y_pred_inverse = y_scaler.inverse_transform(y_pred_reshaped).reshape(n_test_samples, n_outputs)
y_test_inverse = y_scaler.inverse_transform(y_test_reshaped).reshape(n_test_samples, n_outputs)

y_test_flat = y_test_inverse.flatten()
y_pred_flat = y_pred_inverse.flatten()

175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [ ]:
# Compute Metrics for Each Time Step

for i in range(n_outputs):
    y_true = y_test_inverse[:, i]
    y_pred = y_pred_inverse[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # Mean Absolute Percentage Error (MAPE)
    # Avoid division by zero by adding a small epsilon to y_test_flat if necessary
    epsilon = 1e-10
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%")



Time Step 1 Evaluation Metrics:
RMSE: 37.0549
MAE: 24.9980
MAPE: 28.06%

Time Step 2 Evaluation Metrics:
RMSE: 41.8798
MAE: 28.5677
MAPE: 31.32%

Time Step 3 Evaluation Metrics:
RMSE: 45.6936
MAE: 31.6238
MAPE: 35.18%

Time Step 4 Evaluation Metrics:
RMSE: 48.4740
MAE: 33.8421
MAPE: 40.00%

Time Step 5 Evaluation Metrics:
RMSE: 50.9437
MAE: 35.7679
MAPE: 44.62%

Time Step 6 Evaluation Metrics:
RMSE: 52.8113
MAE: 37.5645
MAPE: 52.58%


# Code to predict next 6 steps step-by-step

#### We will use
* the last 12 steps

* previous one week (24 steps)

* previous one month  (168 steps)

*  to forecast current (0 step)

## 1. Create input and output data

In [ ]:
# Create input-output sequences with the provided function
X_train, y_train, X_train_df, y_train_df = create_multi_step_sequence(train_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_valid, y_valid, X_valid_df, y_valid_df = create_multi_step_sequence(valid_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)
X_test, y_test, X_test_df, y_test_df = create_multi_step_sequence(test_set, last_n_steps=12, day_lag=24, week_lag=168, n_future_steps=1)

In [ ]:
X_train.shape, X_train, y_train.shape, y_train

((19570, 14, 1),
 array([[[173.],
         [168.],
         [155.],
         ...,
         [ 27.],
         [ 21.],
         [ 15.]],
 
        [[168.],
         [155.],
         [186.],
         ...,
         [  9.],
         [  8.],
         [  9.]],
 
        [[155.],
         [186.],
         [333.],
         ...,
         [ 10.],
         [ 10.],
         [  9.]],
 
        ...,
 
        [[141.],
         [142.],
         [107.],
         ...,
         [160.],
         [129.],
         [131.]],
 
        [[142.],
         [107.],
         [128.],
         ...,
         [ 94.],
         [ 87.],
         [ 77.]],
 
        [[107.],
         [128.],
         [150.],
         ...,
         [ 80.],
         [ 63.],
         [ 35.]]]),
 (19570, 1),
 array([[ 9.],
        [10.],
        [ 8.],
        ...,
        [94.],
        [80.],
        [63.]]))

## 2. Normalise the data after split (step-by-step)

Normalise X

In [ ]:
# Separate scalers for inputs and outputs
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape x_train to 2D for scaling
n_samples, n_timesteps, n_features = X_train.shape
x_train_reshaped = X_train.reshape(-1, n_features)  # Shape: (n_samples * n_timesteps, n_features)
# Fit the scaler on the training data
x_scaler.fit(x_train_reshaped)
# Transform the training data
x_train_scaled = x_scaler.transform(x_train_reshaped)
# Reshape back to original shape
x_train_scaled = x_train_scaled.reshape(n_samples, n_timesteps, n_features)

# x_val
n_val_samples = X_valid.shape[0]
x_val_reshaped = X_valid.reshape(-1, n_features)
x_val_scaled = x_scaler.transform(x_val_reshaped)
x_val_scaled = x_val_scaled.reshape(n_val_samples, n_timesteps, n_features)

# x_test
n_test_samples = X_test.shape[0]
x_test_reshaped = X_test.reshape(-1, n_features)
x_test_scaled = x_scaler.transform(x_test_reshaped)
x_test_scaled = x_test_scaled.reshape(n_test_samples, n_timesteps, n_features)

Normalise y

In [ ]:
# Reshape y_train to 2D for scaling
y_train_reshaped = y_train.reshape(-1, 1)  # Shape: (n_samples * n_outputs, 1)
# Fit the scaler on the training data
y_scaler.fit(y_train_reshaped)
# Transform the training data
y_train_scaled = y_scaler.transform(y_train_reshaped)
# Reshape back to original shape
y_train_scaled = y_train_scaled.reshape(n_samples, y_train.shape[1])

# y_val
y_val_reshaped = y_valid.reshape(-1, 1)
y_val_scaled = y_scaler.transform(y_val_reshaped)
y_val_scaled = y_val_scaled.reshape(n_val_samples, y_valid.shape[1])

# y_test
y_test_reshaped = y_test.reshape(-1, 1)
y_test_scaled = y_scaler.transform(y_test_reshaped)
y_test_scaled = y_test_scaled.reshape(n_test_samples, y_test.shape[1])

## 3. Build the CNN model (step-by-step)

In [ ]:
def create_cnn_model_recursive(input_shape,
                               dropout_rate=0.5,
                               learning_rate=0.001,
                               filters=64,
                               kernel_size=3):
    model = keras.Sequential([
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape),
        layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(dropout_rate),
        layers.Flatten(),
        layers.Dense(50, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(1)  # Output layer for one-step prediction
    ])
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

## 4. Define Separate Hyperparameter Grids

In [ ]:
from itertools import product

In [ ]:
dropout_rates = [0, 0.3, 0.5]
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]
filters_list = [32, 64, 128]
kernel_sizes = [2, 3, 5]

# Create a list of all hyperparameter combinations
hyperparameter_combinations = list(product(dropout_rates,
                                           learning_rates, batch_sizes,
                                           filters_list,
                                           kernel_sizes))


In [ ]:
best_val_mae = np.inf
best_hyperparams = None
best_model = None

for idx, (dropout_rate, learning_rate, batch_size,
          filters, kernel_size) in enumerate(hyperparameter_combinations):
    print(f"\nCombination {idx+1}/{len(hyperparameter_combinations)}")
    print(f"Training with dropout_rate={dropout_rate}, "
          f"learning_rate={learning_rate}, batch_size={batch_size}, "
          f"filters={filters}, "
          f"kernel_size={kernel_size}")

    model = create_cnn_model_recursive(
        input_shape=(n_timesteps, 1),
        dropout_rate=dropout_rate,
        learning_rate=learning_rate,
        filters=filters,
        kernel_size=kernel_size
    )

    # Initialize EarlyStopping inside the loop
    early_stopping = EarlyStopping(
        monitor='val_mae',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )

    # Train the model
    history = model.fit(
        x_train_scaled, y_train_scaled,
        epochs=50,
        batch_size=batch_size,
        validation_data=(x_val_scaled, y_val_scaled),
        callbacks=[early_stopping],
        verbose=0  # Set to 1 to see detailed training output
    )

    # Get the best validation MAE from this training run
    val_mae = min(history.history['val_mae'])
    print(f"Validation MAE: {val_mae:.4f}")

    # Update best model if current one is better
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_model = model
        best_hyperparams = {
            'dropout_rate': dropout_rate,
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'filters': filters,
            'kernel_size': kernel_size
        }

print("\nBest Hyperparameters:")
for param, value in best_hyperparams.items():
    print(f"{param}: {value}")
print(f"Best Validation MAE: {best_val_mae:.4f}")


Combination 1/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=32, kernel_size=2


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 20: early stopping
Restoring model weights from the end of the best epoch: 10.
Validation MAE: 0.0400

Combination 2/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=32, kernel_size=3
Epoch 17: early stopping
Restoring model weights from the end of the best epoch: 7.
Validation MAE: 0.0404

Combination 3/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=32, kernel_size=5
Epoch 24: early stopping
Restoring model weights from the end of the best epoch: 14.
Validation MAE: 0.0408

Combination 4/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=64, kernel_size=2
Epoch 31: early stopping
Restoring model weights from the end of the best epoch: 21.
Validation MAE: 0.0407

Combination 5/243
Training with dropout_rate=0, learning_rate=0.01, batch_size=32, filters=64, kernel_size=3
Epoch 15: early stopping
Restoring model weights from the end of the best epoch: 5.
Validation MAE: 0.0413

Combination 6/243
Tra

Validation MAE of 0.0385/0.0383

Combination 280/370/

Dropout Rate 1 (dropout_rate_1): 0/0

Dropout Rate 2 (dropout_rate_2): 0/0

Learning Rate (learning_rate): 0.001/0.001

Batch Size (batch_size): 32/64

Filters 1 (filters_1): 64/64

Filters 2 (filters_2): 64/128

Kernel Size 1 (kernel_size_1): 2/2

Kernel Size 2 (kernel_size_2): 2/2

Summary of Combination 370:

In [ ]:
best_hyperparams

{'dropout_rate': 0,
 'learning_rate': 0.001,
 'batch_size': 32,
 'filters': 128,
 'kernel_size': 3}

In [ ]:
def recursive_forecast(model, input_seq, n_steps, x_scaler, y_scaler):
    """
    Perform recursive forecasting using the trained model.

    Parameters:
    - model: Trained model
    - input_seq: The initial input sequence (scaled), shape (n_timesteps, n_features)
    - n_steps: Number of future steps to predict
    - x_scaler: Scaler used for input features
    - y_scaler: Scaler used for target variable

    Returns:
    - predictions: List of predicted values (in original scale)
    """
    predictions = []
    current_input = input_seq.copy()  # Shape: (n_timesteps, n_features)

    for _ in range(n_steps):
        # Reshape to (1, n_timesteps, n_features)
        input_cnn = current_input.reshape((1, current_input.shape[0], current_input.shape[1]))

        # Predict the next time step (scaled)
        yhat_scaled = model.predict(input_cnn, verbose=0)  # Shape: (1, 1)

        # Inverse transform the prediction to original scale
        yhat = y_scaler.inverse_transform(yhat_scaled)[0, 0]

        # Append prediction to the list
        predictions.append(yhat)

        # Scale the predicted value to match input feature scaling
        yhat_scaled_for_input = x_scaler.transform(yhat_scaled)  # Shape: (1, n_features)

        # Update the input sequence
        current_input = np.vstack((current_input[1:], yhat_scaled_for_input))

    return predictions


In [ ]:
# Number of steps to predict
n_steps = 6

# Initialize lists to store predictions and actual values
all_predictions = []
all_actuals = []

# Ensure we have enough data for recursive predictions
n_test_samples = x_test_scaled.shape[0]
for i in range(n_test_samples - n_steps):
    # Get the input sequence for the current sample
    input_seq = x_test_scaled[i]  # Shape: (n_timesteps, n_features)

    # Perform recursive forecasting
    predictions = recursive_forecast(
        model=best_model,
        input_seq=input_seq,
        n_steps=n_steps,
        x_scaler=x_scaler,
        y_scaler=y_scaler
    )

    # Get the actual future values (in original scale)
    actual_values = y_test[i+1:i + n_steps + 1].flatten()

    # Store the predictions and actual values
    all_predictions.append(predictions)
    all_actuals.append(actual_values)

# Convert lists to numpy arrays
all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

In [ ]:
epsilon = 1e-10  # For MAPE calculation to avoid division by zero

# Loop over each time step
for i in range(n_steps):
    y_true = all_actuals[:, i]
    y_pred = all_predictions[:, i]

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_true, y_pred)

    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # Mean Absolute Percentage Error (MAPE)
    y_true_safe = np.where(y_true == 0, epsilon, y_true)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100

    print(f"\nTime Step {i+1} Evaluation Metrics:")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")



Time Step 1 Evaluation Metrics:
MAE: 29.5153
RMSE: 41.4575
MAPE: 31.99%

Time Step 2 Evaluation Metrics:
MAE: 39.7577
RMSE: 56.8027
MAPE: 41.80%

Time Step 3 Evaluation Metrics:
MAE: 56.7176
RMSE: 76.7950
MAPE: 56.77%

Time Step 4 Evaluation Metrics:
MAE: 119.2168
RMSE: 138.3477
MAPE: 90.02%

Time Step 5 Evaluation Metrics:
MAE: 125.7082
RMSE: 144.7713
MAPE: 97.36%

Time Step 6 Evaluation Metrics:
MAE: 123.4105
RMSE: 142.7198
MAPE: 93.86%
